## Step1:Handling BED file format for easy processing

In [ ]:
# Process it into a format that is convenient for subsequent generation of the seq sequence
import re

# read id list
def gtf2bed(id_list, gtf_file, custom_bed_file):

    with open(id_list, 'r') as f:
        crispr_ids = set(line.strip() for line in f)

    #  Process the GTF file.
    def extract_attribute(attr_str, key):
        match = re.search(f'{key} "([^"]+)"', attr_str)
        return match.group(1) if match else None

    # Convert to a BED file.
    with open(gtf_file, 'r') as f_in, \
         open(custom_bed_file, 'w') as f_out:
        for line in f_in:
            if line.startswith('#'):
                continue
            
            fields = line.strip().split('\t')
            if len(fields) < 9:
                continue
                
            chr_name = fields[0]
            start = str(int(fields[3]) - 1)  # BED 0-based
            end = fields[4]
            feature_type = fields[2]
            strand = fields[6]
            attributes = fields[8]

            gene_name = extract_attribute(attributes, 'gene_name')
            
            if gene_name not in crispr_ids:
                continue
            if gene_name == 'LINC00869' and gene_id == 'ENSG00000277147.1':
                continue  # 跳过不处理
            if feature_type == 'transcript':
                id_field = extract_attribute(attributes, 'transcript_id')
            elif feature_type == 'exon':
                id_field = extract_attribute(attributes, 'transcript_id') 
            else:  # gene
                id_field = extract_attribute(attributes, 'gene_id')
            gene_id = extract_attribute(attributes, 'gene_id')
            if gene_id.startswith('LH'):
                bed_line = f"{chr_name}\t{start}\t{end}\t{gene_id}-{id_field}\t0\t{strand}\n"
            else:
                bed_line = f"{chr_name}\t{start}\t{end}\t{gene_name}-{id_field}\t0\t{strand}\n"
            
            f_out.write(bed_line)
if __name__ == "__main__":
    gtf2bed('./test/crispri_id.txt', './test/lncrna.gtf', './test/temp_seq/crispri_temp.bed')
    gtf2bed('./test/crispr_delete_id.txt', './test/gencode.v19.long_noncoding_RNAs.gtf', './test/temp_seq/crispr_delete_temp.bed')
    gtf2bed('./test/crispr_splice_id.txt', './test/gencode.v20.long_noncoding_RNAs.gtf', './test/temp_seq/splice.bed')


In [ ]:
import os

# flow,convert to hg38
os.system('../liftOver ./test/temp_seq/crispri_temp.bed ../hg19ToHg38.over.chain.gz ./test/temp_seq/i38_temp.bed ./test/output/unmap_crispri.bed')
os.system('../liftOver ./test/temp_seq/crispr_delete_temp.bed ../hg19ToHg38.over.chain.gz ./test/temp_seq/delete_temp.bed ./test/output/unmap_crispr_delete.bed')
os.system('grep -v -E "LH00477|LH02126|LH14878" ./test/temp_seq/i38_temp.bed > ./test/temp_seq/i38.bed')
os.system('rm -f ./test/temp_seq/i38_temp.bed ./test/temp_seq/crispri_temp.bed ./test/temp_seq/crispr_delete_temp.bed')


In [ ]:
import pandas as pd

def get_group_key(nameid):
        return nameid
# find transcript
def identify_transcript(group):
    return group.iloc[(group[2] - group[1]).argmax()]

def process_group(group):

    transcript = identify_transcript(group)
    # print(transcript)

    exons = group[group.index != transcript.name].copy()
    
    if len(exons) == 0:
        return None
        

    rel_starts = []
    lengths = []
    

    exons = exons.sort_values(by=[1])
    
    for _, exon in exons.iterrows():
        rel_start = exon[1] - transcript[1]
        length = exon[2] - exon[1]
        rel_starts.append(str(rel_start))
        lengths.append(str(length))
    
    result_row = transcript.copy()
    result_row[6] = ','.join(rel_starts)
    result_row[7] = ','.join(lengths)
    
    return pd.DataFrame([result_row])

def process_bed_file(bed_file, output_bed_file):
# input the hg38 bed file
    df = pd.read_csv(bed_file, sep='\t', header=None)
    df['group_key'] = df[3].apply(get_group_key)
    # print(df['group_key'].head(20))

    results = []
    for _, group in df.groupby('group_key'):
        result = process_group(group)
        if result is not None:
            results.append(result)

    result = pd.concat(results)
    result = result.drop('group_key', axis=1)
    result.to_csv(output_bed_file, sep='\t', header=False, index=False)
process_bed_file('./test/temp_seq/i38.bed', './test/temp_seq/seq_i38.bed')
process_bed_file('./test/temp_seq/delete_temp.bed', './test/temp_seq/seq_delete38.bed')
process_bed_file('./test/temp_seq/splice.bed', './test/temp_seq/seq_splice38.bed')
process_bed_file('./test/output/crispr_casrx38.bed', './test/temp_seq/seq_casrx38.bed')
os.system('rm -f ./test/temp_seq/i38.bed ./test/temp_seq/delete_temp.bed ./test/temp_seq/splice.bed ')


## Step2: Generate sequence files at the gene level.

In [ ]:
!python get_seq.py

## Step3: Trim and generate sequences

In [ ]:
import pandas as pd

def parse_fasta_header(header):
    fields = header[1:].split(':')
    # print(fields)
    return {
        'gene_id': fields[0],
        'start': int(fields[2]) - 1,
        'end': int(fields[3]),
        'strand': fields[4],
    }

def get_transcript_seq(gene_seq, rel_starts, lengths, strand):
    """Obtain transcript sequences."""
    if strand == '-':
        gene_seq = gene_seq[::-1]  
    
    exon_seqs = []
    starts = [int(x) for x in rel_starts.split(',')]
    lens = [int(x) for x in lengths.split(',')]
    
    for start, length in zip(starts, lens):
        exon_seq = gene_seq[start:start+length]
        exon_seqs.append(exon_seq)
    
    transcript_seq = ''.join(exon_seqs)
    
    if strand == '-':
        transcript_seq = transcript_seq[::-1]  
    return transcript_seq

def process_files(fa_file, bed_file):
  
    gene_seqs = {}
    current_gene = None
    current_info = None
    
    with open(fa_file) as f:
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                current_info = parse_fasta_header(line)
                current_gene = current_info['gene_id']
                gene_seqs[current_gene] = {'seq': '', 'info': current_info}
            else:
                gene_seqs[current_gene]['seq'] += line
    
   
    df = pd.read_csv(bed_file, sep='\t', header=None)
    
    with open('./test/output/lncRNA2.fa', 'a') as outf:
        for _, row in df.iterrows():
            transcript_id = row[3].rsplit('-',1)[1]
            gene_id = row[3].rsplit('-',1)[0]
            if gene_id in gene_seqs:
                transcript_seq = get_transcript_seq(
                    gene_seqs[gene_id]['seq'],
                    row[6],
                    row[7],
                    row[5]
                )
                outf.write(f">{transcript_id}\n{transcript_seq}\n")

if __name__ == '__main__':
    # seq_splice.bed is the file generated in the previous step.
    process_files(fa_file='./test/output/crispr_gene.fa', bed_file='./test/temp_seq/seq_delete38.bed')
    process_files(fa_file='./test/output/crispr_gene.fa', bed_file='./test/temp_seq/seq_i38.bed')
    process_files(fa_file='./test/output/crispr_gene.fa', bed_file='./test/temp_seq/seq_splice38.bed')
    process_files(fa_file='./test/output/crispr_gene.fa', bed_file='./test/temp_seq/seq_casrx38.bed')